In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms
 
from torch.utils.data import DataLoader,Dataset



In [15]:
TRAIN_DATA = '../data/train.csv'
TEST_DATA = '../data/test.csv'
TRAIN_IMG = '../data/train_test_data/train/'

In [3]:
train_df = pd.read_csv(TRAIN_DATA)
train_df.head()

,label,latitude,longitude,year,example_path
0,0,-2.051853,111.826093,2001,train_test_data/train/1297.png
1,2,-1.989349,105.309496,2013,train_test_data/train/1199.png
2,0,1.223256,100.702217,2014,train_test_data/train/1348.png
3,0,-2.342948,103.890226,2008,train_test_data/train/2214.png
4,0,-0.126555,101.758175,2011,train_test_data/train/2220.png


In [16]:
test_df = pd.read_csv(TEST_DATA)
test_df.head()

,latitude,longitude,year,example_path
0,0.761681,122.755954,2006,train_test_data/test/69.png
1,-8.059785,113.053791,2007,train_test_data/test/469.png
2,-2.006610,111.746316,2002,train_test_data/test/6.png
3,0.901765,114.042495,2016,train_test_data/test/351.png
4,1.911210,100.829633,2008,train_test_data/test/1001.png


In [11]:
img = Image.open(TRAIN_IMG+'1199.png')
px.imshow(np.asarray(img))

In [14]:
img.mode

'RGB'

In [36]:
class ImageDataset(Dataset):
  def __init__(self,df,data_folder,transform):
    self.df = df
    self.transform = transform
    self.img_folder = data_folder
     
    self.image_names = self.df[:]['example_path']
    self.labels = self.df[:]['label']
   
  def __len__(self):
    return len(self.image_names)
 
  def __getitem__(self,index):
     
    image=Image.open(self.img_folder+self.image_names.iloc[index])

    image=self.transform(image)
    targets=self.labels[index]
     
    sample = {'image': image,'labels':targets}
 
    return sample

train_transform = transforms.Compose([
                       transforms.ToTensor()])

train_data = ImageDataset(train_df, '../data/', train_transform)
train_DL = DataLoader(train_data, batch_size=4, shuffle=True)
sample = next(iter(train_DL))
print(sample['labels'][0])
px.imshow(sample['image'][0].numpy().transpose((1, 2, 0)))


tensor(1)
